# Wzór TensorFlow dla głębokiej sieci neuronowej

W tym przykładzie opisuję części deep neural network na przykładzie klasycznej numerycznej bazy MNIST.
MNIST to baza ręcznie pisanych cyfr w zakresie 0-9 z którą praktycznie każdy w branży miał kiedyś styczność.
Planuję zastosować ten wzornik do swoich celów później, gdyż otrzymałem podobne zadanie testowe.
Miej na względzie, że ten wzór nie pokazuje w jaki sposób preprocesować przygotować, preprocesować, ani dzielić bazę danych na zbiory (treningowy/walidacyjny/testowy), a jedynie pokazuje jak zbudować algorytm sieci deep.

#### 0. Obejście przestarzałego kodu bazy MNIST
Jak spróbowałem użyć prostem wersji z tutoriala, to dostałem info, że jest przestarzała i zaraz wycofują.
Poniższe obejście powinno załatwić sprawę. Kiedyś wystarczała do tego 1 linijka kodu.
Teraz podziwiajcie co się porobiło... Znalezione na oficjalnej stronie twórców TF, gdzie prowadził kod błędu.

Poniższy kod to de-facto official/mnist.py, o którym mowa jak się próbuje używać przestarzałej funkcji read_data_sets. Ogólnie używa się tego kodu, który swego czasu już próbowałem rozpracować, a który używał znajomy hindus, którego kod analizuję w folderze "spyder" niniejszego repozytorium. Są do skrypty "board.py i importowane do niego "input_data". Postaram się to tutaj przeanalizować jeszcze raz i bardziej szczegółowo, bo widać
trudno będzie tego kawałka uniknąć. 

Pamiętaj, że w takich sytuacjach możesz zawsze zrobić to co zrobił kolega z Indii i po prostu podzielić kod na dwie części, a następnie zaimportować sobie jedną na początku kodu. Na pewno uporządkuje Ci do miejsce pracy, co jest cenne zwłaszcza w Jupyterze, gdzie zeszyty bez importowania mają tentencję do ciągnięcia się kilometrami.

Jeszcze zobaczę na ile jest to absolutnie konieczne, a na ile można to zostawić jako ciekawostkę w stylu "obchodzimy przestarzały MNIST". Na razie skupię się na kodzie głównym tak, aby reszta działała bez większych problemów.


#### Uwaga: Alternatywne źródło danych MNIST
Jak ostatnio sprawdzałem, to dane ze strony Yann wciąż działają, ale w linijce 57 wrzuciłem alternatywne źródło. 

Nie rozbijam kodu i nie wrzucam tego komentarza celowo, żeby nie robić bałaganu, ale chcę, żebyś zwrócił(a) na to uwagę.

In [1]:
# W komentarzu poniżej oryginalna licencja. 
# Jest to standardowa licencja Apache 2.0., która dodatkowo mówi nam ni mniej ni więcej tyle, 
# że twórcy TF nie odpowiadają za użytkowanie przez nas tego obejścia.
# Wszelkie zmiany do tego kodu wymuszone były na mnie tym, że inaczej nie działał. ;)
# Komentarze w języku polskim dla lepszego zrozumienia działania mechanizmu, jeśli się za to wezmę.

#  Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip

import numpy as np
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin

from tensorflow.python.framework import dtypes
from tensorflow.python.framework import random_seed
from tensorflow.python.platform import gfile

import collections
import csv
import os
from os import path
import random
import tempfile
import time

from six.moves import urllib

from tensorflow.contrib.framework import deprecated
from tensorflow.python.platform import gfile

# W tym miejscu definiujemy inne źródło danych MNIST. Strona Yann w komentarzu.
# OStatnio jak sprawdzałem działała.
# CVDF mirror of http://yann.lecun.com/exdb/mnist/
DEFAULT_SOURCE_URL = 'https://storage.googleapis.com/cvdf-datasets/mnist/'

Dataset = collections.namedtuple('Dataset', ['data', 'target'])
Datasets = collections.namedtuple('Datasets', ['train', 'validation', 'test'])


def load_csv_with_header(filename,
                         target_dtype,
                         features_dtype,
                         target_column=-1):
  """Load dataset from CSV file with a header row."""
  with gfile.Open(filename) as csv_file:
    data_file = csv.reader(csv_file)
    header = next(data_file)
    n_samples = int(header[0])
    n_features = int(header[1])
    data = np.zeros((n_samples, n_features), dtype=features_dtype)
    target = np.zeros((n_samples,), dtype=target_dtype)
    for i, row in enumerate(data_file):
      target[i] = np.asarray(row.pop(target_column), dtype=target_dtype)
      data[i] = np.asarray(row, dtype=features_dtype)

  return Dataset(data=data, target=target)


def load_csv_without_header(filename,
                            target_dtype,
                            features_dtype,
                            target_column=-1):
  """Load dataset from CSV file without a header row."""
  with gfile.Open(filename) as csv_file:
    data_file = csv.reader(csv_file)
    data, target = [], []
    for row in data_file:
      target.append(row.pop(target_column))
      data.append(np.asarray(row, dtype=features_dtype))

  target = np.array(target, dtype=target_dtype)
  data = np.array(data)
  return Dataset(data=data, target=target)


def shrink_csv(filename, ratio):
  """Create a smaller dataset of only 1/ratio of original data."""
  filename_small = filename.replace('.', '_small.')
  with gfile.Open(filename_small, 'w') as csv_file_small:
    writer = csv.writer(csv_file_small)
    with gfile.Open(filename) as csv_file:
      reader = csv.reader(csv_file)
      i = 0
      for row in reader:
        if i % ratio == 0:
          writer.writerow(row)
        i += 1


def load_iris(data_path=None):
  """Load Iris dataset.
  Args:
      data_path: string, path to iris dataset (optional)
  Returns:
    Dataset object containing data in-memory.
  """
  if data_path is None:
    module_path = path.dirname(__file__)
    data_path = path.join(module_path, 'data', 'iris.csv')
  return load_csv_with_header(
      data_path,
      target_dtype=np.int,
      features_dtype=np.float)


def load_boston(data_path=None):
  """Load Boston housing dataset.
  Args:
      data_path: string, path to boston dataset (optional)
  Returns:
    Dataset object containing data in-memory.
  """
  if data_path is None:
    module_path = path.dirname(__file__)
    data_path = path.join(module_path, 'data', 'boston_house_prices.csv')
  return load_csv_with_header(
      data_path,
      target_dtype=np.float,
      features_dtype=np.float)


def retry(initial_delay,
          max_delay,
          factor=2.0,
          jitter=0.25,
          is_retriable=None):
  """Simple decorator for wrapping retriable functions.
  Args:
    initial_delay: the initial delay.
    factor: each subsequent retry, the delay is multiplied by this value.
        (must be >= 1).
    jitter: to avoid lockstep, the returned delay is multiplied by a random
        number between (1-jitter) and (1+jitter). To add a 20% jitter, set
        jitter = 0.2. Must be < 1.
    max_delay: the maximum delay allowed (actual max is
        max_delay * (1 + jitter).
    is_retriable: (optional) a function that takes an Exception as an argument
        and returns true if retry should be applied.
  """
  if factor < 1:
    raise ValueError('factor must be >= 1; was %f' % (factor,))

  if jitter >= 1:
    raise ValueError('jitter must be < 1; was %f' % (jitter,))

  # Generator to compute the individual delays
  def delays():
    delay = initial_delay
    while delay <= max_delay:
      yield delay * random.uniform(1 - jitter,  1 + jitter)
      delay *= factor

  def wrap(fn):
    """Wrapper function factory invoked by decorator magic."""

    def wrapped_fn(*args, **kwargs):
      """The actual wrapper function that applies the retry logic."""
      for delay in delays():
        try:
          return fn(*args, **kwargs)
        except Exception as e:  # pylint: disable=broad-except)
          if is_retriable is None:
            continue

          if is_retriable(e):
            time.sleep(delay)
          else:
            raise
      return fn(*args, **kwargs)
    return wrapped_fn
  return wrap


_RETRIABLE_ERRNOS = {
    110,  # Connection timed out [socket.py]
}


def _is_retriable(e):
  return isinstance(e, IOError) and e.errno in _RETRIABLE_ERRNOS


@retry(initial_delay=1.0, max_delay=16.0, is_retriable=_is_retriable)
def urlretrieve_with_retry(url, filename=None):
  return urllib.request.urlretrieve(url, filename)


def maybe_download(filename, work_directory, source_url):
  """Download the data from source url, unless it's already here.
  Args:
      filename: string, name of the file in the directory.
      work_directory: string, path to working directory.
      source_url: url to download from if file doesn't exist.
  Returns:
      Path to resulting file.
  """
  if not gfile.Exists(work_directory):
    gfile.MakeDirs(work_directory)
  filepath = os.path.join(work_directory, filename)
  if not gfile.Exists(filepath):
    temp_file_name, _ = urlretrieve_with_retry(source_url)
    gfile.Copy(temp_file_name, filepath)
    with gfile.GFile(filepath) as f:
      size = f.size()
    print('Successfully downloaded', filename, size, 'bytes.')
  return filepath

def _read32(bytestream):
  dt = numpy.dtype(numpy.uint32).newbyteorder('>')
  return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]


def extract_images(f):
  """Extract the images into a 4D uint8 numpy array [index, y, x, depth].
  Args:
    f: A file object that can be passed into a gzip reader.
  Returns:
    data: A 4D uint8 numpy array [index, y, x, depth].
  Raises:
    ValueError: If the bytestream does not start with 2051.
  """
  print('Extracting', f.name)
  with gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2051:
      raise ValueError('Invalid magic number %d in MNIST image file: %s' %
                       (magic, f.name))
    num_images = _read32(bytestream)
    rows = _read32(bytestream)
    cols = _read32(bytestream)
    buf = bytestream.read(rows * cols * num_images)
    data = numpy.frombuffer(buf, dtype=numpy.uint8)
    data = data.reshape(num_images, rows, cols, 1)
    return data


def dense_to_one_hot(labels_dense, num_classes):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot


def extract_labels(f, one_hot=False, num_classes=10):
  """Extract the labels into a 1D uint8 numpy array [index].
  Args:
    f: A file object that can be passed into a gzip reader.
    one_hot: Does one hot encoding for the result.
    num_classes: Number of classes for the one hot encoding.
  Returns:
    labels: a 1D uint8 numpy array.
  Raises:
    ValueError: If the bystream doesn't start with 2049.
  """
  print('Extracting', f.name)
  with gzip.GzipFile(fileobj=f) as bytestream:
    magic = _read32(bytestream)
    if magic != 2049:
      raise ValueError('Invalid magic number %d in MNIST label file: %s' %
                       (magic, f.name))
    num_items = _read32(bytestream)
    buf = bytestream.read(num_items)
    labels = numpy.frombuffer(buf, dtype=numpy.uint8)
    if one_hot:
      return dense_to_one_hot(labels, num_classes)
    return labels


class DataSet(object):

  def __init__(self,
               images,
               labels,
               fake_data=False,
               one_hot=False,
               dtype=dtypes.float32,
               reshape=True,
               seed=None):
    """Construct a DataSet.
    one_hot arg is used only if fake_data is true.  `dtype` can be either
    `uint8` to leave the input as `[0, 255]`, or `float32` to rescale into
    `[0, 1]`.  Seed arg provides for convenient deterministic testing.
    """
    seed1, seed2 = random_seed.get_seed(seed)
    # If op level seed is not set, use whatever graph level seed is returned
    numpy.random.seed(seed1 if seed is None else seed2)
    dtype = dtypes.as_dtype(dtype).base_dtype
    if dtype not in (dtypes.uint8, dtypes.float32):
      raise TypeError(
          'Invalid image dtype %r, expected uint8 or float32' % dtype)
    if fake_data:
      self._num_examples = 10000
      self.one_hot = one_hot
    else:
      assert images.shape[0] == labels.shape[0], (
          'images.shape: %s labels.shape: %s' % (images.shape, labels.shape))
      self._num_examples = images.shape[0]

      # Convert shape from [num examples, rows, columns, depth]
      # to [num examples, rows*columns] (assuming depth == 1)
      if reshape:
        assert images.shape[3] == 1
        images = images.reshape(images.shape[0],
                                images.shape[1] * images.shape[2])
      if dtype == dtypes.float32:
        # Convert from [0, 255] -> [0.0, 1.0].
        images = images.astype(numpy.float32)
        images = numpy.multiply(images, 1.0 / 255.0)
    self._images = images
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0

  @property
  def images(self):
    return self._images

  @property
  def labels(self):
    return self._labels

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_completed(self):
    return self._epochs_completed

  def next_batch(self, batch_size, fake_data=False, shuffle=True):
    """Return the next `batch_size` examples from this data set."""
    if fake_data:
      fake_image = [1] * 784
      if self.one_hot:
        fake_label = [1] + [0] * 9
      else:
        fake_label = 0
      return [fake_image for _ in xrange(batch_size)], [
          fake_label for _ in xrange(batch_size)
      ]
    start = self._index_in_epoch
    # Shuffle for the first epoch
    if self._epochs_completed == 0 and start == 0 and shuffle:
      perm0 = numpy.arange(self._num_examples)
      numpy.random.shuffle(perm0)
      self._images = self.images[perm0]
      self._labels = self.labels[perm0]
    # Go to the next epoch
    if start + batch_size > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Get the rest examples in this epoch
      rest_num_examples = self._num_examples - start
      images_rest_part = self._images[start:self._num_examples]
      labels_rest_part = self._labels[start:self._num_examples]
      # Shuffle the data
      if shuffle:
        perm = numpy.arange(self._num_examples)
        numpy.random.shuffle(perm)
        self._images = self.images[perm]
        self._labels = self.labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size - rest_num_examples
      end = self._index_in_epoch
      images_new_part = self._images[start:end]
      labels_new_part = self._labels[start:end]
      return numpy.concatenate(
          (images_rest_part, images_new_part), axis=0), numpy.concatenate(
              (labels_rest_part, labels_new_part), axis=0)
    else:
      self._index_in_epoch += batch_size
      end = self._index_in_epoch
      return self._images[start:end], self._labels[start:end]


def read_data_sets(train_dir,
                   fake_data=False,
                   one_hot=False,
                   dtype=dtypes.float32,
                   reshape=True,
                   validation_size=5000,
                   seed=None,
                   source_url=DEFAULT_SOURCE_URL):
  if fake_data:

    def fake():
      return DataSet(
          [], [], fake_data=True, one_hot=one_hot, dtype=dtype, seed=seed)

    train = fake()
    validation = fake()
    test = fake()
    return Datasets(train=train, validation=validation, test=test)

  if not source_url:  # empty string check
    source_url = DEFAULT_SOURCE_URL

  TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
  TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
  TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
  TEST_LABELS = 't10k-labels-idx1-ubyte.gz'

  local_file = maybe_download(TRAIN_IMAGES, train_dir,
                                   source_url + TRAIN_IMAGES)
  with gfile.Open(local_file, 'rb') as f:
    train_images = extract_images(f)

  local_file = maybe_download(TRAIN_LABELS, train_dir,
                                   source_url + TRAIN_LABELS)
  with gfile.Open(local_file, 'rb') as f:
    train_labels = extract_labels(f, one_hot=one_hot)

  local_file = maybe_download(TEST_IMAGES, train_dir,
                                   source_url + TEST_IMAGES)
  with gfile.Open(local_file, 'rb') as f:
    test_images = extract_images(f)

  local_file = maybe_download(TEST_LABELS, train_dir,
                                   source_url + TEST_LABELS)
  with gfile.Open(local_file, 'rb') as f:
    test_labels = extract_labels(f, one_hot=one_hot)

  if not 0 <= validation_size <= len(train_images):
    raise ValueError('Validation size should be between 0 and {}. Received: {}.'
                     .format(len(train_images), validation_size))

  validation_images = train_images[:validation_size]
  validation_labels = train_labels[:validation_size]
  train_images = train_images[validation_size:]
  train_labels = train_labels[validation_size:]

  options = dict(dtype=dtype, reshape=reshape, seed=seed)

  train = DataSet(train_images, train_labels, **options)
  validation = DataSet(validation_images, validation_labels, **options)
  test = DataSet(test_images, test_labels, **options)

  return Datasets(train=train, validation=validation, test=test)


def load_mnist(train_dir='MNIST-data'):
  return read_data_sets(train_dir)

## Tutaj zaczynamy właściwą pracę - import bibliotek i bazy danych
Jak wyżej, ściągamy istotne dla nas biblioteki i odpalamy bazę ściągniętą z archiwów internetu
wysłużoną bazę MINST. Zwróć uwagę na komentarze. Postarałem się w tej sekcji wyjaśnić możliwie dużo.

In [2]:
# Ściągamy biblioteki operacyjne. Jak nie wiesz co to jest cofnij się do poprzednich zeszytów kursowych.
import numpy as np
import tensorflow as tf

# Poniższy kod ściąga bazę MNIST i przyporządkowuje ją do zmiennej.
# Jest to duże ułatwienie na poziomie szkolenia względem tego co robiono kiedyś, 
# a co możesz sobie obejrzeć w kodzie działającym właściwie identycznie w folderze "spyder"
# od którego rozpocząłem swoją analizę. Z czasem społeczność stworzyła szereg ułatwień
# i paczka startowa samouczka jest przewidziana jako opcjonalny download ze strony twórcy.
# Dane są już preprocesowane, oraz rozbite na treningowe, walidacyjne i testowe, 
# zatem i o to nie musisz się tym razem martwić. 
# PRZESTARZAŁY KOD: from tensorflow.examples.tutorials.mnist import input_data
# UWAGA: Do powyższego kodu zastosowano obejście stworzone przez twórców TF. (Ta wielka kobyła powyżej)
# Używanie go pomimo ostrzeżeń o wycofywaniu grozi błędem i w ogóle... :P

# Read_data_sets ma dwa argumenty. Pierwszy mówi funkcji o lokalizacji danych a drugi o sposobie
# przyporządkowywania (kodowania pozycji) zmiennych. One_hot jest dosyć popularne, gdyż sprawdza się przy
# niewielkiej liczbie argumentów i nie generuje niepotrzebnych korelacji między zmiennymi.
# Jeśli jednak masz dużo zmiennych, to warto zastosować binary (czyli one_hot="False"), gdyż posiadanie
# np. kilkuset, lub kilkunastu tysięcy zmiennych kodujących pozycję będzie dość zasobożerne...
mnist = read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 1.  Rysujemy i trenujemy model
Tak jak w poprzednich przykładach. Możesz porównać z zeszytami 1 i 2.

In [10]:
### PANEL GŁÓWNY ###
lr = 0.001              # Prędkość uczenia się modelu. Domyślna: 0.001
bs = 100                # Wielkość partii w jakich uczymy model (patrz linijka 93 - batch_size)
n_epochs = 15           # Górne organiczenie ilości "epok", tj. przejść przez bazę danych modelu (linijka 101 - max_epochs)
input_size = 784        # Rozmiar danych wejściowych to 784, gdyż rozmiar obrazka w bazie danych MNIST to 28x28px=784px.
output_size = 10        # Rozmiar danych wyjściowych to 10, gdyż baza danych MNIST zawiera 10 cyfr od 0 do 9
                        # z których model wybiera właściwe odpowiedzi z pewnym prawdopodobieństwem dla każdej.
hidden_layer_size = 50  # rozmiar każdej ukrytej warstwy "neuronów".
# Założyliśmy tutaj dość często używaną metodykę, że wszystkie ukryte warstwy mają tę samą "grubość". Domyślna: 50.
# W zależności jednak od problemu, który chcesz rozwiązać, możesz używać warstw różnej wielkości. Spróbuj np. 100.

tf.reset_default_graph() # Resetuje zmienne zapisane w pamięci modelu z jego poprzednich przebiegów.
# Jest to o tyle konieczne, że deklarujemy nasze zmienne jako globalne jako część domyślnego (default) modelu,
# i niewykonanie tej operacji może nam namieszać, kiedy dopracowujemy model w ramach eksperymentów.

# Podstawniki jak w zeszycie 2.
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

# Używamy metody tf.get_variable("nazwa_zmiennej",[kształt,zmiennej]) aby zdefiniować Wagi i Obciążenia
# dla każdego połączenia. W tym wypadku jest to np. Wagi_1 (z wejściowych do 1 ukrytej warstwy) =
# = tf.get_variable("wagi_1", [rozmiar_danych_wejściowych np. 784, rozmiar_warstwy ukrytej np. 50])
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
# outputs_1 to wynik aktywacji w ramach wybranej przez nas funkcji aktywacji (tutaj wybralismy dość popularną Relu)
# Możesz wypróbować inne, jak sigmoid, ale tutaj raczej zobaczysz, że będzie to skutkowało powolnym uczeniem się.
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

# Druga ukryta warstwa ma rozmiar ukryta_warstwa x ukryta warstwa, co jest logiczne, czyż nie? ;)
# Wszystkie kolejne ukryte warstwy będą miały taki rozmiar, poza ostatnią, którą będziemy "mnożyć"
# przez rozmiar danych wyjściowych.
weights_2 = tf.get_variable("weights_2",[hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])
# Zwróć uwagę z czego zwracamy wynik outputs_2, czyli wynik aktywacji drugiej warstwy.
outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

# W tym przykładzie używam dwóch ukrytych warstw, mimo, że jest to ilość suboptymalna, 
# po prostu, żeby pokazać jak działają. Możesz zmienne dodawać ręcznie, 
# albo, do czego zachęcam, zrobić sobie funkcję, która będzie dodawać (albo co lepsze recyclingować jak u Hindusa) 
# kolejne warstwy za Ciebie.

# Wagi łączące ostatnią warstwę ukrytą z warstwą danych wyjściowych (outputs).
weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

# Zwróć uwagę, że nie zwróciliśmy tutaj od razu funkcji aktywacyjnej, a jedynie iloczyn skalarny + obciążenie 3.

outputs = tf.matmul(outputs_2, weights_3) + biases_3

# Poniższa funkcja aplikuje zarówno Aktywator Softmax, który jest wskazany przy danych wyjściowych
# (bo daje wyskalowane przedziały ufności dzielone przez dostępne opcje sumarycznie do 1)
# oraz aplikuje koszt z funkcji logarytmicznej liczony od całkowitego błędu. 
# Format użycia to: tf.nn.softmax_cross_entropy_with_logits(logits,labels) Tutaj: logits = wyjściowe, labels = cele (targets)
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets)

# Poniższa metoda poprawia nam osiągi. W tej sposób TF szuka nam średnią elementów tensora w danym wymiarze.
# Więcej o zastosowaniu ENG tutaj*
mean_loss = tf.reduce_mean(loss)

# Wybieramy model treningowy. Ponieważ poszliśmy do przodu z teorią, to tym razem wybierzemy sobie ADAM,
# który jest o tyle dobry, że raczej nie zacina się na lokalnym (fałszywym) minimum funkcji i optymalnie szuka
# globalnego minimum. Jest to metoda z 2015, więc dość nowa. Daj mi znać jeśli wymyślono coś lepszego.
optimize = tf.train.AdamOptimizer(learning_rate=lr).minimize(mean_loss)

# Metoda tf.argmax pozwala nam wybrać która zmienna ma najwyższy "wynik" (patrz niżej) zwracany przez funkcję w sposób: 
# (outputs: z aktywatora softmax, tj. tutaj model próbuje "zgadnąć" prawidłowy wynik o który nam chodzi,
# przydzielając poszczególnym zmiennym prawdopodobieństwo. Agrmax wybiera najwyższe prawdopodobieństwo.)
# (targets: zdefiniowane przez nas w ramach bazy "labels", do czego dąży model. 
# W wypadku "OneHot" tylko jedna zmienna będzie miała wartość 1 a pozostałe 0 i to ją wybierze Argmax.)
# tf.equal (bool) porównuje te dwa ciągi, które zwracają argmax i jeśli się pokrywają zwraca 1 a jeśli nie 0.
# Otrzymujemy więc wektor zero-jedynkowy, który nabierze sensu w ramach kolejnej funkcji...
out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

# tf.reduce_mean*, o którym możesz poczytać poniżej pozwala nam logicznie obliczyć trafność modelu.
# Po prostu bierze wszystkie te 0 (nietrafione) i 1 (trafione decyzje modelu) i wyciąga średnią. Oto nasza trafność.:)
# tf.cast(obiekt_do_konwersji, docelowy_format) to nasze ubezpieczenie, na wszelki wypadek
# aby dane, które zwróci nam funkcja na pewno były w odpowiednim formacie. (tutaj tf.float32)
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

# Odpalamy sesję i inicjalizujemy zmienne w ramach sesji. Szczegóły w zeszycie 2, pkt 5 i 6.
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

# Wielkość pakietów. Zdecyduj o wielkości pakietów na które będzie dzielona baza treningowa.
# Tzn. jak często model będzie poprawiał swoje wagi i obciążenia.
# Jeśli wpiszesz ilość równą ilości obserwacji (niezalecane)
# model przejdzie w ramach 1 epoki na raz przez całą bazę danych i będzie najdokładniejszy 
# (przykład: "Gradient Descent"), ale jest to bardzo zasobochłonne
# Im mniejszy pakiet względem bazy danych, tym częstsza aktualizacja w ramach przechodzenia przez nią,
# ale mniejsza dokładność (Jak w Stochastic Gradient Descent).
# Ogólnie warto się z tym pobawić i raczej na ogół warto poświęcić trochę dokładności na rzecz szybkości.
# Startowa : 100 mówi, że w ramach jednego "przejścia" przez bazę, 
# wagi i średnie zostaną dostosowane (ilość_przykładów / 100 razy). 
# Dzielenie "floordiv" zaokrągla ilość do równej wartości. // Edit: Wyciągnięta na panel główny.
batch_size = bs
# Ta zmienna to wynik dzielenia liczby przykładów treningowych w bazie danych przez rozmiar partii.
batches_number = mnist.train._num_examples // batch_size

# Poniżej dwa mechanizmy zatrzymujące:
# Ta zmienna ogranicza z góry liczbę "przejść" przez bazę, tak aby model się nie zaciął. // Edit: Wyciągnięta na panel główny.
max_epochs = n_epochs
# Ta zmienna da nam pewność, że model nie przerwie działania po pierwszym przebiegu (epoce).
# Model porównuje wyniki treningowe do bazy walidacyjnej i sprawdza, czy nie zaszło tzw. overfitting (naddopasowanie modelu)
# Wysoka wartość poprzedniej straty z walidacj przeciwdziała tutaj przerwaniu po pierwszej epoce,
# gdyż na początku model takowej wartości nie ma, więc domyślne 0 spowodowałoby niewłaściwe przerwanie, co mija się z celem...
prev_validation_loss = 9999999.

# Podobną pętlę masz w zeszycie 2. Jak coś jest niejasne sprawdź tam, bo staram się nie powtarzać.
# W tym miejscu odbywa się uczenie modelu. Zauważ, że ta pętla nie przebiegnie więcej niż 15 razy.
for epoch_counter in range(max_epochs):
    
    # Ustalamy zmienną do której pętla zapisuje obecną wartość funkcji straty.
    curr_epoch_loss = 0.
    
    # Pętla kręci się do wykorzystania wszystkich partii.
    for batch_counter in range(batches_number):
        
        # Tutaj pętla ładuje [100] wejść (input_batch) i [100] celów (target_batch) ([100] wynika z "batch_size")
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        
        # Tutaj optymalizuje algorytm i kalkuluje funkcję straty dla każdej partii. 
        _, batch_loss = sess.run([optimize, mean_loss], 
            feed_dict={inputs: input_batch, targets: target_batch})
        
        # Tutaj zapisuje stratę dla obecnej iteracji pętli
        curr_epoch_loss += batch_loss
    
    # Teraz zmienna curr_epoch_loss faktycznie zawiera średnią wartość funkcji straty dla zestawu treningowego
    # dla kolejnej epoki. Otrzymujemy ją dzieląc sumę strat ze wszystkich partii w epoce przez liczbę partii.
    curr_epoch_loss /= batches_number

    # Tutaj ładujemy zestaw walidacyjny wyodrębiony już dla nas w paczce MNIST. Ten zestaw wykorzystujemy
    # wielokrotnie, i podlega on jedynie propagacji w przód, tzn. nie dostosowujemy do niego wag modelu
    # (nie trenujemy na nim). Dzieje się tak dlatego, że względem tej małej próbki (zwykle ok 10% bd)
    # sprawdzamy, czy nie zaszło naddopasowanie (overfitting).
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    
    
    # Oblicza średnią stratę i trafność zestawu walidacyjnego i zapisuje je w odpowiadających im zmiennych
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
                                                    feed_dict={inputs: input_batch, targets: target_batch})
    
    # Zwraca nam w widoczny sposób wartości zmiennych dla każdej 'Epoki'. Formatowanie typowo pythonowskie.
    print('Epoka: '+str(epoch_counter+1)+
          '. Strata z Treningu : '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Strata z Walidacji: '+'{0:.3f}'.format(validation_loss)+
          '. Trafność Walidacji: '+'{0:.2f}'.format(validation_accuracy * 100)+'%')
    
    # Przerywa pętlę wcześniej jeśli strata z walidacji zacznie wzrastać wskazując na naddopasowanie modelu.
    if validation_loss > prev_validation_loss:
        break
        
    # Aktualizuje stratę z walidacji, żeby ją można było porównać po każdej epoce.    
    prev_validation_loss = validation_loss
        
print("Koniec Sesji Treningowej")

/home/SpookyProgrammer/anaconda3/envs/tensorenv/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoka: 1. Strata z Treningu : 0.434. Strata z Walidacji: 0.208. Trafność Walidacji: 93.96%
Epoka: 2. Strata z Treningu : 0.196. Strata z Walidacji: 0.163. Trafność Walidacji: 95.30%
Epoka: 3. Strata z Treningu : 0.148. Strata z Walidacji: 0.134. Trafność Walidacji: 96.08%
Epoka: 4. Strata z Treningu : 0.118. Strata z Walidacji: 0.129. Trafność Walidacji: 96.18%
Epoka: 5. Strata z Treningu : 0.098. Strata z Walidacji: 0.105. Trafność Walidacji: 96.82%
Epoka: 6. Strata z Treningu : 0.083. Strata z Walidacji: 0.099. Trafność Walidacji: 97.06%
Epoka: 7. Strata z Treningu : 0.071. Strata z Walidacji: 0.095. Trafność Walidacji: 97.16%
Epoka: 8. Strata z Treningu : 0.064. Strata z Walidacji: 0.094. Trafność Walidacji: 97.12%
Epoka: 9. Strata z Treningu : 0.055. Strata z Walidacji: 0.097. Trafność Walidacji: 97.12%
Koniec Sesji Treningowej


#### *Metoda tf.reduce_mean - przykłady: 
https://stackoverflow.com/questions/34236252/difference-between-np-mean-and-tf-reduce-mean-in-numpy-and-tensorflow#47242212

# 2. Testujemy Model
Baza testowa, to baza, której nasz model nigdy nie "widział", w przeciwieństwie do bazy walidacyjnej,która pomagała nam uniknąć "naddopasowania". Teraz na takiej "dziewiczej" bazie danych poznamy bliższą prawdziwej wartość Trafności (accuracy) naszego modelu. Czy i na ile mierzy to co miał mierzyć? Przekonajmy się sami. ;)
Wynik może się poprawić lub pogorszyć, ale na ten moment jest to wynik ostateczny.

### UWAGA!!!
Pamiętaj, że jak już raz odpalisz model na bazie testowej, to (zgodnie ze sztuką) nie należy już ponownie poprawiać modelu (dłubać przy hiperparametrach - patrz: model główny). Ta wartość trafności powinna być uznana za ostateczną, bo inaczej (logicznie rzecz biorąc) naddopasowujesz model do bazy testowej, co mija się z celem.

W sytuacji, gdy dalej chcesz dłubać przy modelu, zdobądź nowe próbki do bazy testowej a istniejące wkomponuj w istniejący model (do bazy treningowej/walidacyjnej). Oczywiście baza MNIST jest dla Ciebie bazą treningową, niemniej pamiętaj o tym istotnym założeniu badawczym.

In [12]:
# Kod analogiczny do kodu ładującego i przypisującego zmienne w części 1 do zestawu walidacyjnego,
# więc nie będę się rozpisywał...
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy], 
    feed_dict={inputs: input_batch, targets: target_batch})

# Zmienna test_accuracy, to lista o 1 wartości, z której wyciągamy tę wartość żądając pozycji [0].
# print (test_accuracy)
test_accuracy_percent = test_accuracy[0]

# Formatujemy analogicznie do Trafności walidacji:
print('Trafność Testowa: '+'{0:.2f}'.format(test_accuracy_percent * 100)+'%')

Trafność Testowa: 97.17%


In [13]:
sess.close()